# Seguridad y Privacidad de los Datos - 2024/25

Last edited: 2025/03/25

# 0. Train the model on the provided dataset

In [7]:
import pandas as pd

DATASET_ROUTE = './dataset.csv'

df = pd.read_csv(DATASET_ROUTE)
df



,sha256,label,timestamp,activities_count,providers_count,receivers_count,features_count,intent_filters_count,api_calls_count,urls_count,...,susp_getNetworkCountryIso,susp_getPackageInfo,susp_getSimCountryIso,susp_getSubscriberId,susp_getSystemService,susp_getWifiState,susp_sendTextMessage,susp_setWifiDisabled,susp_setWifiEnabled,susp_system/bin/su
0,B9D5216F9D33FB46B572855F596DB885A9C9A507729AB5...,0,2019-03-04 11:37:03,26,1,13,0,4,8,44,...,0,1,0,0,1,0,0,0,0,0
1,1842246AB160F68456632DBC3B84EFE7F8260AEACEE0C1...,0,2017-03-20 23:01:54,1,1,0,0,1,16,6,...,0,0,0,0,1,0,0,0,0,0
2,DCBEEFAF347DC7B65445F686124D63965BEFA409222C33...,0,2017-01-03 05:38:35,3,0,0,0,1,8,32,...,0,1,0,0,1,0,0,0,0,0
3,6539CF1A61EAEA9B3B448D031FA2714F3F17D3D84E89CB...,0,2019-10-10 01:59:46,11,1,0,0,1,20,27,...,0,1,0,0,1,0,0,0,0,0
4,09A34511FACAFA3DE80D5242C28DBAE875D01503344D41...,0,2017-06-16 03:30:54,3,0,5,0,2,3,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75285,53868176B0207FAB85E45612A1D91D5054662E8E2BA9AB...,0,2017-10-08 01:40:38,53,1,5,0,10,41,101,...,1,1,1,0,1,0,0,0,0,1
75286,FD8230591A83FA11BF6A795FB55DB2D6EDB4AF4ACD98D3...,0,2017-01-25 09:09:37,6,0,0,0,1,12,15,...,0,1,0,0,1,0,0,0,0,0
75287,000D7F02C18405337773937592D3A0ED6EB9BFC75D2F0A...,0,2018-09-07 06:35:17,1,0,0,0,1,20,2,...,0,1,0,0,1,0,0,0,0,0
75288,189E2A17CAE4CD04EA4EA79DC461626DF44DE58D1B3277...,0,2017-04-20 12:26:13,78,0,1,0,4,40,76,...,0,1,1,0,1,0,0,0,0,0


Cositas importantes:

* sha256 poco útil
* Ordenar en orden cronológico y dividir el entrenamiento (entrenar con las de hace 5 años y predecir actuales) dice que es interesante
* 'activities_count', 'providers_count','receivers_count' parecen interesantes según él
* 'api_calls_count' interesante
* 'unused_permissions' permisos pedidos pero no usados
* 'urls_count' cuantas urls se han encontrado dentro de la apk
* Probar varios modelos e ir quitando/poniendo caracteristicas

In [13]:
df.columns

Index(['sha256', 'label', 'timestamp', 'activities_count', 'providers_count',
       'receivers_count', 'features_count', 'intent_filters_count',
       'api_calls_count', 'urls_count',
       ...
       'susp_getNetworkCountryIso', 'susp_getPackageInfo',
       'susp_getSimCountryIso', 'susp_getSubscriberId',
       'susp_getSystemService', 'susp_getWifiState', 'susp_sendTextMessage',
       'susp_setWifiDisabled', 'susp_setWifiEnabled', 'susp_system/bin/su'],
      dtype='object', length=358)

In [10]:
df.value_counts("label")

label
0    67771
1     7519
Name: count, dtype: int64

In [17]:
# removing unused columns
if 'timestamp' in df.columns:
    df = df.drop('timestamp', axis=1)
if 'unused_permissions' in df.columns:
    df = df.drop('unused_permissions', axis=1)
if 'sha256' in df.columns:
    df = df.drop('sha256', axis=1)

df

,label,activities_count,providers_count,receivers_count,features_count,intent_filters_count,api_calls_count,urls_count,services_count,perm_android.permission.ACCESS_ALL_DOWNLOADS,...,susp_getNetworkCountryIso,susp_getPackageInfo,susp_getSimCountryIso,susp_getSubscriberId,susp_getSystemService,susp_getWifiState,susp_sendTextMessage,susp_setWifiDisabled,susp_setWifiEnabled,susp_system/bin/su
0,0,26,1,13,0,4,8,44,14,0,...,0,1,0,0,1,0,0,0,0,0
1,0,1,1,0,0,1,16,6,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,3,0,0,0,1,8,32,0,0,...,0,1,0,0,1,0,0,0,0,0
3,0,11,1,0,0,1,20,27,0,0,...,0,1,0,0,1,0,0,0,0,0
4,0,3,0,5,0,2,3,1,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75285,0,53,1,5,0,10,41,101,5,0,...,1,1,1,0,1,0,0,0,0,1
75286,0,6,0,0,0,1,12,15,0,0,...,0,1,0,0,1,0,0,0,0,0
75287,0,1,0,0,0,1,20,2,0,0,...,0,1,0,0,1,0,0,0,0,0
75288,0,78,0,1,0,4,40,76,1,0,...,0,1,1,0,1,0,0,0,0,0


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56467, 354), (18823, 354), (56467,), (18823,))

In [21]:
# Train the model
rf_model.fit(X_train, y_train)

# Evaluate the model on the training set
train_accuracy = rf_model.score(X_train, y_train)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Evaluate the model on the test set
test_accuracy = rf_model.score(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Training Accuracy: 0.9993
Test Accuracy: 0.9805


# 1. Perform at least one black-box attack and one white-box attack on the model. You may perform additional attacks as desired.

- Black box
- White box

# 2. Prove that your model can be more robust to adversarial examples

Demonstrate that your model maintains high accuracy when subjected to adversarial examples. Provide empirical evidence of robustness by evaluating performance under adversarial attacks.

What Will Be Valued:
- Effective adversarial attack methods and proper defense mechanisms.
- Clear evaluation metrics comparing standard vs. adversarial performance.
- Well-documented methodology and reproducibility of results.

# 3. Try to attack this model

You will be given the anonymous model `android_permission_model.onnx`, previously trained on the provided dataset. 
Analyze the model’s weaknesses by testing different black-box attack strategies.

What Will Be Valued:
- Use of diverse adversarial attack techniques.
- Quantitative analysis of attack effectiveness with relevant metrics.
- Clear documentation of the attack process and reproducibility.

Below, you can find an example on model loading and inference. The model expects every column in the dataset except for `sha256`, `timestamp` and `unused_permissions`.
The model is loaded in an [onnx runtime](https://onnxruntime.ai/docs/api/python/api_summary.html). 

In [5]:
import os
import numpy as np
import pandas as pd
import onnxruntime as rt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

DATASET_ROUTE = './dataset.csv'

df = pd.read_csv(DATASET_ROUTE)
model_path = 'android_permission_model.onnx'

# removing unused columns
if 'timestamp' in df.columns:
    df = df.drop('timestamp', axis=1)
if 'unused_permissions' in df.columns:
    df = df.drop('unused_permissions', axis=1)
if 'sha256' in df.columns:
    df = df.drop('sha256', axis=1)

# features and target separation + train/test split
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

Example inference function using [sess.run()](https://onnxruntime.ai/docs/api/python/api_summary.html#onnxruntime.InferenceSession.run):

In [21]:
def onnx_inference(model_path, test_data):
    """
    Function to load and make predictions with an ONNX model
    """
    # open ONNX inference session
    sess = rt.InferenceSession(model_path)
    
    # input and output names
    input_name = sess.get_inputs()[0].name
    output_names = [output.name for output in sess.get_outputs()]
    
    # send inputs to the model and obtain predictions
    results = sess.run(output_names, {input_name: test_data.astype(np.float32)})

    return results[0]       # results[0] are discrete predicted labels, while results[1] are probabilities

In [22]:
# try and load the model
if not os.path.exists(model_path):
    print(f"Error: Model file '{model_path}' not found!")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Files in current directory: {os.listdir('.')}")
else:
    # small sample to test if it works
    test_sample = X_test.iloc[:5].values
    onnx_predictions = onnx_inference(model_path, test_sample)
    
    if onnx_predictions is not None:
        print("\nONNX Model Predictions (sample):")
        print(onnx_predictions)
        
        # compare with actual labels
        actual_labels = y_test.iloc[:5].values
        print("\nActual labels:")
        print(actual_labels)
    
    # full test set now
    print("\nONNX Model Predictions (full test set):")
    full_predictions = onnx_inference(model_path, X_test.values)

    try:
        print(f"Accuracy: {accuracy_score(y_test, full_predictions):.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, full_predictions))
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, full_predictions))
    except Exception as e:
        print(f"Error during ONNX prediction: {e}")


ONNX Model Predictions (sample):
[1 0 0 0 1]

Actual labels:
[1 0 0 0 0]

ONNX Model Predictions (full test set):
Accuracy: 0.9815

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     16943
           1       0.86      0.97      0.91      1880

    accuracy                           0.98     18823
   macro avg       0.93      0.98      0.95     18823
weighted avg       0.98      0.98      0.98     18823


Confusion Matrix:
[[16642   301]
 [   48  1832]]
